# SQL Server Agent Job Monitoring and Alerting

Alerting and Monitoring may be customised as necessary. This solution may be scheduled locally or on the cloud.

## Using pyodbc

In [ ]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

# database settings should be read from some secure config
server = ''
user = ''
password = ''
database = 'msdb' # ensure that this is msdb unless a stored procedure is used instead

# configure database engine
params = urllib.parse.quote_plus(r'DRIVER={};SERVER={};DATABASE={};UID={};PWD={}'.format(driver, server, database, user, password))
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
conn = create_engine(conn_str)

# alert on failure
def alert(job_name, last_run):
    
    # add method of alert, ie. MS Teams, Slack, API call, etc.
    print('{} failed at {}'.format(job_name, last_run))
    
def sql_monitoring():
    
    # read from existing function on msdb
    df = pd.read_sql('EXEC msdb.dbo.sp_help_jobactivity;', conn)

    # order by last_executed_step_date
    df['last_executed_step_date'] = pd.to_datetime(df['last_executed_step_date']).apply(lambda x: x.date())
    df['last_executed_step_date'] = df['last_executed_step_date'].astype(str)

    # create a dataframe that contains jobs executed today
    df_today = df[df['last_executed_step_date'] == datetime.today().strftime("%Y-%m-%d")]

    # create a dataframe that contains the jobs that have failed
    df_failed = df_today[df_today['run_status'] == 0]

    if len(df_failed) > 0:

        for index, element in df_failed.iterrows():

            alert(element['job_name'], element['last_executed_step_date'])
            
sql_monitoring()

## Using pymssql

In [ ]:
import pandas as pd
import pymssql

# database settings should be read from some secure config
server = ''
user = ''
password = ''
database = 'msdb' # ensure that this is msdb unless a stored procedure is used instead

# configure database engine
conn = pymssql.connect(server, user, password, database)

# alert on failure
def alert(job_name, last_run):
    
    # add method of alert, ie. MS Teams, Slack, API call, etc.
    print('{} failed at {}'.format(job_name, last_run))
    
def sql_monitoring():
    
    # read from existing function on msdb
    df = pd.read_sql('EXEC msdb.dbo.sp_help_jobactivity;', conn)

    # order by last_executed_step_date
    df['last_executed_step_date'] = pd.to_datetime(df['last_executed_step_date']).apply(lambda x: x.date())
    df['last_executed_step_date'] = df['last_executed_step_date'].astype(str)

    # create a dataframe that contains jobs executed today
    df_today = df[df['last_executed_step_date'] == datetime.today().strftime("%Y-%m-%d")]

    # create a dataframe that contains the jobs that have failed
    df_failed = df_today[df_today['run_status'] == 0]

    if len(df_failed) > 0:

        for index, element in df_failed.iterrows():

            alert(element['job_name'], element['last_executed_step_date'])

sql_monitoring()